# Storing Encrypted Data Using Python & SQLite3

In this notebook we will encrypt a plain text file and store it in a database. We will then retrieve the encrypted data from the database and decrypt it using a saved key.

## Introducing cryptography

**Cryptograph** involves creating codes to keep information secret ([techopedia](https://www.techopedia.com/definition/1770/cryptography)). Here, we will make use of **symmetric encryption**. What's that? Consider the diagram below:

```
            message
Alice -------------------> Bob
               |
              Eve

```
Here, Alice (the **sender**) wants to send Bob (the **receiver**) a **message**. This message, however, may be intercepted by Eve (the *eaves*dropping **adversary**). How can Alice and Bob ensure that their message can be securely sent and received?


```
            message
Alice -------------------> Bob
(KEY)          |          (KEY)
              Eve

```

One solution is for Alice and Bob to share some secret **key**. Only those with a correct key can decrypt the contents of *message*. This approach is called *symmetric* encryption because both "sides" of the message, Alice and Bob, share the same key. 

## Preparing a message

The contents of a plain text file (`sample_message.txt`) are printed below:

In [81]:
with open("./sample_message.txt", "r") as f:
    for line in f:
        print(line.strip())

This is a message to be encrypted.
Here is a second message.
Foo bar baz.
FoO bar baz.
Foot boot butt.


This will be our message.

## Encryption with Python

We will make use of Python's [cryptography](https://cryptography.io/en/latest/) to encrypt and decrypt our data.

In [98]:
from cryptography.fernet import Fernet

# You will have to save this key to unencrypt your data later
key = Fernet.generate_key()
fernet = Fernet(key)

token = fernet.encrypt(b"this is a test")
print(f'This is the decrypted string: {fernet.decrypt(token)}')

This is the decrypted string: b'this is a test'


Note the `b` before "this is a string". We have to convert Python 3's usual strings into binary before applying encryption to them. This can be done by opening our plain text file in binary mode.

In [99]:
# Save all lines of our (short) file to memory
binary_lines = []

with open("./sample_message.txt", "rb") as f:
    for line in f:
        binary_lines.append(line)

# Encrypt each of these lines and append them to another list 
encrypted_binary_lines = []

with open("./output.txt", "wb") as f:
    for line in binary_lines:
        encrypted_binary_lines.append(fernet.encrypt(line))

We now have the contents of our file `sample_message.txt` encrypted in our variable `encrypted_binary_lines`.

## Storing encrypted data in a database

Let's score encrypted data in an SQLite3 database (because I'm practicing using them).

Below, while our table `secrets` has two columns, one for a message ID (corresponds to line number) and another for the message itself.

In [100]:
import sqlite3
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Make a table
c.execute('''create table secrets
             (id integer primary key, message text)''')

# Insert our encrypted data into the table
messages = [(n + 1, v) for n, v in enumerate(encrypted_binary_lines)]
c.executemany('insert into secrets values (?,?)', messages)

# Save (commit) the changes and close our connection
conn.commit()
conn.close()

Now, let's access our encrypted data from the SQLite database we just made. First, let's do this using the `sqlite` command on the command line. We'll view our encrypted data this way.

```
sqlite> .tables
secrets
sqlite> .schema secrets
CREATE TABLE secrets
             (id integer primary key, message text);
```

The rows of our table (we'll look at all of them since there aren't so many):

```
sqlite> select * from secrets;
```

This command should return all of our id keys and encrypted messages. Here's what it looked like with my key:

```
0|gAAAAABcT8V1OAOZqxJRc0aZQ48wVqxLKNvtKF8Dg_d1-HE4Zro9RQyhawcxhsxpqFzCzSDmdcnnOoW8bwJX31mS1aDdgS_sOVkfLlniqcmzd093kOOvdbbntuB0eHqbFKK7oCdlHYBz
1|gAAAAABcT8V1mNIafLBG-BG5HOpvGpQpJw4pH9CIkBTyl8vG7Fi4PrjnhB8W-YP7Bn80bb0LIRFTsNa_t6VXr8teKqXbQirAOIo2IKPHzRFloulw1aBIT0Y=
2|gAAAAABcT8V1Fgo-3oSUpr3gpKD1SmWpBAjtvu-59RjHd6ytKzAqJNNzayX8aWQ4phbqqNhxSMZIikHXvoVe_XaddSxRGzQgoQ==
3|gAAAAABcT8V1hk6PmprQBrW8kuZCa8tWtHEdkNzFnCjkV4B3KTFLvA-V5RTBnZQSxhfX5lo4W5Lz2LYDc8UsPT5PhAT3xgUB1g==
4|gAAAAABcT8V1jAblxTZuD4FTClCLzu4IUkoe81L-U2vOhPWfzJwWq9bcpmDnE51jG2Umf29wP39uTksKlcmiwRDRPeJdgMIKmQ==
```

Looks adequately encrypted to me. Let's go decrypt these things using Python and the `fernet` key we made earlier.

In [101]:
# Check to see if our message was stored successfully
conn = sqlite3.connect('example.db')
c = conn.cursor()

c.execute('select * from secrets')
rows = c.fetchall()

for row in rows:
    print(fernet.decrypt(row[1]))

b'This is a message to be encrypted.\n'
b'Here is a second message.\n'
b'Foo bar baz.\n'
b'FoO bar baz.\n'
b'Foot boot butt.'


Cool beans! Our messages, stored encrypted in our database (`example.db`), are now decrypted!

## Saving our key, decrypting our data gain

Before leaving, let's save our key. 

In [104]:
with open ('my_key.txt', 'wb') as f:
    f.write(key)

Our key is now stored in the file `my_key.txt`. We can load the data from this file to acess our encrypted data later.

In [106]:
with open('my_key.txt', 'rb') as f:
    fernet_key_returns = Fernet(f.read())
    
    conn = sqlite3.connect('example.db')
    c = conn.cursor()
    
    # Read a line from database
    c.execute('select * from secrets where id=?', (1,))
    msg = c.fetchone()
    
    print(fernet_key_returns.decrypt(msg[1]))

b'This is a message to be encrypted.\n'


## Summary

Here, without getting into the difficult details of what cryptography is and how it is done, we encrypted some data and stored it in a SQLite3 database. Then, we retrieved this data using a saved key.

If you were feeling sneaky, you could give your friend a copy of this key and send them encrypted data in a file like `example.db`. Even if some people broke into your private email server and stole `example.db` from you or your friend, without the key, they could not read the contents of `example.db`. Whooaaaa...!

## Other Notes

Here, we stored encrypted messages within an unencypted database. It looks like the [SQLite Encryption Extension](https://www.sqlite.org/see/doc/trunk/www/readme.wiki) (SEE) is a way to encrypt the entire database.

To run this code (again), you may have to delete the `secrets` table previously created. Connect to `example.db` using SQLite:

```
$ sqlite3 example.db
```

...and then drop the `secrets` table.

```
sqlite> drop table secrets;
```